# Adding All

In [1]:
macro select_force(type)
    # Parameters
    μ = (:μ)
    r_max = (:r_max)
    r_min = (:r_min)
    α = (:α)

    # List of forces functions
    if type == (:cubic)
        struct_select = quote
            Base.@kwdef struct $type
                μ::Float64
                r_max::Float64
                r_min::Float64
                f::Function = r -> ifelse.(
                    r.<=$r_max, 
                    -$μ .* (r .- r_max).^2 .* (r .- r_min), 
                    0.0
                )
            end
        end
    elseif type == (:GLS)
        struct_select = quote
            Base.@kwdef struct $type
                μ::Float64
                r_max::Float64
                r_min::Float64
                α :: Float64
                f::Function = r -> ifelse.(
                    r.<=$r_min,  
                    -$μ .* log(1 .+ r .- $r_min),
                    ifelse.(
                        r.<=$r_max,
                        -$μ .* (r .- $r_min) .* exp.(-α .* (r .- r_min)),
                        0 
                    )
                )
            end
        end
    else
        struct_select = quote
            println("FORCE FUNCTION NOT FOUNDED")
        end
    end
    return esc(:($struct_select))
end

@select_force (macro with 1 method)

## ERROR

In [2]:
@select_force ESTE_NO

FORCE FUNCTION NOT FOUNDED


## Cubic

In [3]:
@select_force cubic
force = cubic(μ = 1, r_max = 2, r_min = 1)
force = cubic(μ = 1, r_max = 2, r_min = 2)
dump(force)

cubic
  μ: Float64 1.0
  r_max: Float64 2.0
  r_min: Float64 2.0
  f: #3 (function of type var"#3#5"{Int64, Int64, Int64})
    μ: Int64 1
    r_max: Int64 2
    r_min: Int64 2


In [4]:
force.f(1.2)

0.5120000000000001

In [5]:
force.f(2 .* rand(3,3) .+ 1)

3×3 Matrix{Float64}:
 0.258034   0.00955375  0.0
 0.0        0.566878    0.0
 0.0744524  0.0         0.0

## GLS

In [6]:
@select_force GLS
force = GLS(μ = 1, r_max = 2, r_min = 1, α = 2)
dump(force)

GLS
  μ: Float64 1.0
  r_max: Float64 2.0
  r_min: Float64 1.0
  α: Float64 2.0
  f: #8 (function of type var"#8#10"{Int64, Int64, Int64, Int64})
    μ: Int64 1
    r_max: Int64 2
    r_min: Int64 1
    α: Int64 2


In [7]:
force.f(1.2)

-0.13406400920712783

In [8]:
force.f(2 .* rand(3,3) .+ 1)

3×3 Matrix{Real}:
 -0.158939  -0.176221  -0.107928
 -0.15565    0          0
  0          0          0

## Iterator

In [9]:
Base.@kwdef struct ModelParameters
    Force::Dict
    Time::Dict
    Neighbor::Dict
    Geometry::Dict
    Coding::Dict
end

ModelParameters

In [10]:
Conditions = ModelParameters(
    Force = Dict(
        "μ"      => 1:2, 
        "r_max"  => 2:3,
        "r_min"  => 3,
        "f_p"    => 4:5
    ),
    Time = Dict(
        "T_f" => 100000,
        "dt"  => 0.1
    ),
    Neighbor = Dict(
        "n_knn" => 50,
        "nn"    => 12
    ),
    Geometry = Dict(
        "R_agg"   => 10,
        "num_agg" => 2
    ),
    Coding  = Dict(
        "n_text" => 200
    )
);

In [11]:
# Testing all the conditions of the fusion
Iter_Time = Conditions.Time["T_f"]
Iter_Forc = [Conditions.Force[i] for i in ["r_max","f_p","μ"]]
for k in Iterators.product(Iter_Time, Iter_Forc...)
    # Initial Data
    R_agg = Conditions.Geometry["R_agg"]
    r_min = Conditions.Force["r_min"]
    X = Float32.(readdlm("../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu
    
    # Generating the Path
    if num_agg == 1
        Path = "../../results/OneAgg/R_Agg($(R_agg))/T_$(k[1])/fp_($(k[3]))/K_($(k[4]))"
    else
        Path = "../../results/TwoAgg/R_Agg($(R_agg))/T_$(k[1])/fp_($(k[3]))/K_($(k[4]))"
    end
    File = "tf_($(k[1]))|s=($r_min)|fp_($(k[3]))|K_($(k[4]))|rmax_$(k[2])|_GPU.xyz"

    for p in 4:8 # p is the position every folder on the path
        if !isdir(join(split(Path, "/")[1:p],"/")) 
            mkdir(join(split(Path, "/")[1:p],"/")) 
        end
    end

    # Calculating all above
    println("---------------------------------------------------------------")
    if num_agg == 1
        println("Calculating T_Final=$(k[1]) | R_Max = $(k[2])) | fp = $(k[3])) | k = $(k[4]) \n for $(Conditions.Geometry["num_agg"]) Aggregate")
    else
        println("Calculating T_Final=$(k[1]) | R_Max = $(k[2])) | fp = $(k[3])) | k = $(k[4]) \n for $(Conditions.Geometry["num_agg"]) Aggregates")
    end

    if File in readdir(Path)
        if countlines(Path*"/"*File) < (2*size(X,1)+2)*(Conditions.Coding["n_text"]+1)
            println("Calculated with less data. Recalculating")
            rm(Path*"/"*File)
            if num_agg == 1
                one_aggregate(
                    Path*"/"*File,
                    true,
                    Conditions.Coding["n_text"],
                    k[1], k[2], k[3], k[4], 
                    R_agg
                )
            else
                fusion(
                    Path*"/"*File,
                    Conditions.Coding["n_text"],
                    true,
                    k[1], k[2], k[3], k[4], 
                    R_agg
                )
            end
        else
            println("This is already calculated")           
        end
    else
        if num_agg == 1
            one_aggregate(
                Path*"/"*File,
                true,
                Conditions.Coding["n_text"],
                k[1], k[2], k[3], k[4], 
                R_agg
            )
        else
            fusion(
                Path*"/"*File,
                true,
                Conditions.Coding["n_text"],
                k[1], k[2], k[3], k[4], 
                R_agg
            )
        end
    end
end

LoadError: UndefVarError: readdlm not defined